<a href="https://colab.research.google.com/github/DomizianoScarcelli/big-data-project/blob/main/NN_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install dependencies

In [ ]:
!pip install petastorm -qq
!pip install lightning -qq
!pip install pyspark -qq
!pip install -U -q PyDrive -qq
!apt install openjdk-8-jdk-headless -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.4/562.4 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
openjdk-8-jdk-headless is already the n

In [ ]:
#@title Imports
import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly

import pyspark
from pyspark.sql import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, FloatType, LongType
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.ml.linalg import SparseVector, DenseVector

from tqdm.notebook import tqdm
import time
import gc

from google.colab import drive

import pytorch_lightning as pl
from petastorm import make_batch_reader
from petastorm.pytorch import DataLoader

In [ ]:
#@title Set up variables
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
GDRIVE_DIR = "/content/drive"
GDRIVE_HOME_DIR = GDRIVE_DIR + "/MyDrive"
GDRIVE_DATA_DIR = GDRIVE_HOME_DIR + "/Big Data/datasets"
DATASET_FILE = os.path.join(GDRIVE_DATA_DIR, "pyspark_friendly_spotify_playlist_dataset")
AUDIO_FEATURES_FILE = os.path.join(GDRIVE_DATA_DIR, "pyspark_track_features")
LITTLE_SLICE_FILE = os.path.join(GDRIVE_DATA_DIR, "little_slice")
SMALL_SLICE_FLIE = os.path.join(GDRIVE_DATA_DIR, "small_slice")
LITTLE_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "little_slice_audio_features")
MICRO_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "micro_slice_audio_features")
SPLITTED_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "splitted_pyspark_track_features")
SAVED_DFS_PATH = os.path.join(GDRIVE_DATA_DIR, "saved_dfs")
RANDOM_SEED = 42 # for reproducibility
os.environ["JAVA_HOME"] = JAVA_HOME
os.environ["PYSPARK_PYTHON"]="python"

In [ ]:
drive.mount(GDRIVE_DIR, force_remount=True)

Mounted at /content/drive


In [ ]:
#@title Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '12G').\
                set('spark.driver.memory', '12G').\
                set('spark.driver.maxResultSize', '100G').\
                set("spark.executor.extraJavaOptions", "-XX:+UseG1GC").\
                setAppName("PySparkTutorial").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

## Setup ngrok

In [ ]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=70953b6345089854df609c684d4a8a85b0b721e03d70b7cfd96aae21b568c420
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [ ]:
!ngrok authtoken 2NVN8kdoOnMVtlDGGWtwsbT5M3Q_2EJv2HE77FEXkz978Qtnq

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from pyngrok import ngrok

# Open a ngrok tunnel on the port 4050 where Spark is running
port = '4050'
public_url = ngrok.connect(port).public_url

In [ ]:
print("To access the Spark Web UI console, please click on the following link to the ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

To access the Spark Web UI console, please click on the following link to the ngrok tunnel "https://abc7-107-167-177-9.ngrok-free.app" -> "http://127.0.0.1:4050"


In [ ]:
#@title Check if everything is ok
spark, sc._conf.getAll()


(<pyspark.sql.session.SparkSession at 0x7f9841592da0>,
 [('spark.executor.extraJavaOptions',
   '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false -XX:+UseG1GC'),
  ('spark.app.name', 'PySparkTutorial'),
  ('spark.driver.host', '74431fbd7482'),


# Data acquisition

In [ ]:

song_schema = StructType([
    StructField("pos", IntegerType(), True),
    StructField("artist_name", StringType(), True),
    StructField("track_uri", StringType(), True),
    StructField("artist_uri", StringType(), True),
    StructField("track_name", StringType(), True),
    StructField("album_uri", StringType(), True),
    StructField("duration_ms", LongType(), True),
    StructField("album_name", StringType(), True)
])

playlist_schema = StructType([
    StructField("name", StringType(), True),
    StructField("collaborative", StringType(), True),
    StructField("pid", IntegerType(), True),
    StructField("modified_at", IntegerType(), True),
    StructField("num_tracks", IntegerType(), True),
    StructField("num_albums", IntegerType(), True),
    StructField("num_followers", IntegerType(), True),
    StructField("tracks", ArrayType(song_schema), True),
    StructField("num_edits", IntegerType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("num_artists", IntegerType(), True),
])

audio_features_schema = StructType([
    StructField("danceability", FloatType(), True),
    StructField("energy", FloatType(), True),
    StructField("key", IntegerType(), True),
    StructField("loudness", FloatType(), True),
    StructField("mode", IntegerType(), True),
    StructField("speechiness", FloatType(), True),
    StructField("acousticness", FloatType(), True),
    StructField("instrumentalness", FloatType(), True),
    StructField("liveness", FloatType(), True),
    StructField("valence", FloatType(), True),
    StructField("tempo", FloatType(), True),
    StructField("type", StringType(), True),
    StructField("id", StringType(), True),
    StructField("uri", StringType(), True),
    StructField("track_href", StringType(), True),
    StructField("analysis_url", StringType(), True),
    StructField("duration_ms", LongType(), True),
    StructField("time_signature", IntegerType(), True)
])


In [ ]:
slice_df = spark.read.schema(playlist_schema).json(SMALL_SLICE_FLIE, multiLine=True)

# Import dataframe inside DataLoader

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
PLAYLIST_EMBEDDINGS = os.path.join(SAVED_DFS_PATH, "playlist_embeddings.json")

In [ ]:
mapped_slice_df = spark.read.json(PLAYLIST_EMBEDDINGS)

In [ ]:
import numpy as np
from pyspark.sql.functions import udf
from pyspark.ml.linalg import VectorUDT
import torch
from typing import Tuple, List


def convert_sparse_to_indices(df: DataFrame) -> DataFrame:
  """
  Given a dataframe fo columns "pos":int and "tracks":SparseVector, it returns a new dataframe where
  the SparseVector are replaced with a list of the indices where the values are.
  (The value information is lost, but we don't care since they are binary values so they will be all ones)
  """

  SparseTupleType = ArrayType(
    ArrayType(IntegerType())
  )

  @udf(returnType=SparseTupleType)
  def transform_array(item: SparseVector):
    """
    Given a SparseVector (binary) it returns the tuple that represent it, of the type (size, indices)
    """
    return [item.indices]

  embeddings_df = df.withColumn("embedding_indices", transform_array(col("tracks"))).drop("tracks")
  return embeddings_df


In [ ]:
pytorch_df = convert_sparse_to_indices(mapped_slice_df.select("pid", "tracks"))

In [ ]:
pytorch_df.show()

+----+--------------------+
| pid|          embeddings|
+----+--------------------+
|1000|[[3345, 3386, 452...|
|1001|[[21674, 30886, 4...|
|1002|[[7381, 9005, 215...|
|1003|[[1667, 1681, 336...|
|1004|[[210, 810, 1282,...|
|1005|[[1666, 10041, 10...|
|1006|[[1235, 2470, 763...|
|1007|[[2455, 5450, 577...|
|1008|[[1479, 3330, 805...|
|1009|[[2546, 3285, 646...|
|1010|[[850, 1254, 2481...|
|1011|[[73, 1266, 2973,...|
|1012|[[1764, 3714, 730...|
|1013|[[429, 1258, 1681...|
|1014|[[136, 3715, 4561...|
|1015|[[50287, 57102, 7...|
|1016|[[835, 1515, 2142...|
|1017|[[20910, 69451, 7...|
|1018|[[812, 1252, 2778...|
|1019|[[2088, 3665, 409...|
+----+--------------------+
only showing top 20 rows



In [ ]:
def convert_indices_to_sparse_tensor(indices: List[int]) -> torch.sparse.FloatTensor:
    """
    Given a list of strictly increasing indices, it returns the sparse binary torch tensor that they represent.
    """
    indices = torch.LongTensor(indices)
    SIZE = 681806
    arr = np.zeros(SIZE)
    arr[indices] = np.ones(len(indices))
    return torch.tensor(arr).to_sparse()

tensor(indices=tensor([[1, 2, 3, 4]]),
       values=tensor([1., 1., 1., 1.]),
       size=(681806,), nnz=4, dtype=torch.float64, layout=torch.sparse_coo)

In the paper they have two matrices,l et $n$ be the number of unique songs, $m$ the number of playlists and $k$ the number of unique artists:

- $P \in \mathbb{R}^{m \times n}$ where $p_i = 1$ if song $i$ is in the playlist, $p_i=0$ otherwise
- $A \in \mathbb{R}^{m \times k}$ where $a_i=1$ if the artist is present in the playlist, $a_i = 0$ otherwise

In [ ]:
from petastorm.spark import SparkDatasetConverter, make_spark_converter
from petastorm.unischema import dict_to_spark_row, Unischema, UnischemaField
from petastorm.codecs import ScalarCodec, CompressedImageCodec, NdarrayCodec
from torchvision import transforms
from petastorm import TransformSpec


CACHE= os.path.join(GDRIVE_DIR, "/big_data/cache")
spark.conf.set(SparkDatasetConverter.PARENT_CACHE_DIR_URL_CONF, f'file://{CACHE}')


# DfSchema = Unischema('DfSchema', [
#     UnischemaField('pos', np.uint8, (), ScalarCodec(IntegerType()), False),
#     UnischemaField('embedding_indices', np.float32, (2,), NdarrayCodec(), False),
# ])

pytorch_df = convert_sparse_to_indices(mapped_slice_df.select("pid", "tracks"))
converter = make_spark_converter(pytorch_df)

with converter.make_torch_dataloader() as dataloader:
  for item in dataloader:
    print(item)
    break

{'pid': tensor([51000, 51001, 51002, 51003, 51004, 51005, 51006, 51007, 51008, 51009,
        51010, 51011, 51012, 51013, 51014, 51015, 51016, 51017, 51018, 51019,
        51020, 51021, 51022, 51023, 51024, 51025, 51026, 51027, 51028, 51029,
        51030, 51031]), 'modified_at': tensor([1483228800, 1480636800, 1491609600, 1492300800, 1507766400, 1508716800,
        1494460800, 1508889600, 1509062400, 1500940800, 1477785600, 1509148800,
        1475193600, 1507593600, 1472688000, 1466294400, 1466812800, 1490140800,
        1445472000, 1509062400, 1449532800, 1480204800, 1466640000, 1448409600,
        1427241600, 1509235200, 1440979200, 1382400000, 1478304000, 1445472000,
        1504569600, 1495065600])}


In [ ]:
embeddings = pytorch_df.first().embedding_indices

# PyTorch Model

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pickle


class DAE_tied(nn.Module):
    def __init__(self, conf):
        super(DAE_tied, self).__init__()
        self.save_dir = conf["save"]

        self.n_batch = conf["batch"]
        self.n_input = conf["n_input"]
        self.n_hidden = conf["hidden"]
        self.learning_rate = conf["lr"]
        self.reg_lambda = conf["reg_lambda"]

        self.x_positions = torch.LongTensor()
        self.x_ones = torch.FloatTensor()

        self.y_positions = torch.LongTensor()
        self.y_ones = torch.FloatTensor()

        self.keep_prob = torch.tensor(conf["keep_prob"], dtype=torch.float32)
        self.input_keep_prob = torch.tensor(conf["input_keep_prob"], dtype=torch.float32)

        self.weights = {}
        self.biases = {}
        self.d_params = []

    def init_weight(self):
        self.weights['encoder_h'] = nn.Parameter(torch.FloatTensor(self.n_input, self.n_hidden))
        nn.init.xavier_uniform_(self.weights['encoder_h'])
        self.biases['encoder_b'] = nn.Parameter(torch.FloatTensor(self.n_hidden))
        nn.init.zeros_(self.biases['encoder_b'])
        self.biases['decoder_b'] = nn.Parameter(torch.FloatTensor(self.n_input))
        nn.init.zeros_(self.biases['decoder_b'])
        self.d_params = [self.weights['encoder_h'], self.biases['encoder_b'], self.biases['decoder_b']]

    # Building the encoder
    def encoder(self, x):
        # Encoder Hidden layer with sigmoid activation #1
        layer = torch.add(torch.matmul(x, self.weights['encoder_h']), self.biases['encoder_b'])
        layer = torch.sigmoid(layer)
        layer = torch.nn.functional.dropout(layer, p=1 - self.keep_prob)

        return layer

    # Building the decoder
    def decoder(self, x):
        # Decoder Hidden layer with sigmoid activation #1
        layer = torch.sigmoid(torch.add(torch.matmul(x, self.weights['encoder_h'].t()), self.biases['decoder_b']))
        return layer

    def l2_loss(self):
        l2 = torch.nn.functional.l2_loss(self.weights['encoder_h']) + torch.nn.functional.l2_loss(
            self.biases['decoder_b']) + torch.nn.functional.l2_loss(self.biases['encoder_b'])
        return l2

    def forward(self, x_positions, x_ones, y_positions, y_ones):
        self.x_positions = x_positions
        self.x_ones = x_ones
        self.y_positions = y_positions
        self.y_ones = y_ones

        x_sparse = torch.sparse.FloatTensor(self.x_positions.t(), self.x_ones, torch.Size([self.n_batch, self.n_input]))
        self.x = x_sparse.to_dense()
        y_sparse = torch.sparse.FloatTensor(self.y_positions.t(), self.y_ones, torch.Size([self.n_batch, self.n_input]))
        self.y = y_sparse.to_dense()

        x_dropout = torch.nn.functional.dropout(self.x, p=1 - self.input_keep_prob)
        reduce_sum = torch.sum(x_dropout, dim=1, keepdim=True)
        self.x_dropout = torch.div(x_dropout, reduce_sum + 1e-10)

        encoder_op = self.encoder(self.x_dropout)
        self.y_pred = self.decoder(encoder_op)

        l2 = self.l2_loss()

        L = -torch.sum(self.y * torch.log(self.y_pred + 1e-10) +
                       0.55 * (1 - self.y) * torch.log(1 - self.y_pred + 1e-10), dim=1)
        self.cost = torch.mean(L) + self.reg_lambda * l2

    def save_model(self):
        params = [param.detach().numpy() for param in self.d_params]
        with open(self.save_dir, 'wb') as f:
            pickle.dump(params, f)

            
class DAE(DAE_tied):
    def __init__(self, conf):
        super(DAE, self).__init__(conf)
        self.initval_dir = conf["initval"]

    def init_weight(self):
        if self.initval_dir == 'NULL':
            self.weights['encoder_h'] = nn.Parameter(torch.FloatTensor(self.n_input, self.n_hidden))
            nn.init.xavier_uniform_(self.weights['encoder_h'])
            self.weights['decoder_h'] = nn.Parameter(torch.FloatTensor(self.n_input, self.n_hidden))
            nn.init.xavier_uniform_(self.weights['decoder_h'])
            self.biases['encoder_b'] = nn.Parameter(torch.FloatTensor(self.n_hidden))
            nn.init.zeros_(self.biases['encoder_b'])
            self.biases['decoder_b'] = nn.Parameter(torch.FloatTensor(self.n_input))
            nn.init.zeros_(self.biases['decoder_b'])
        else:
            with open(self.initval_dir, 'rb') as f:
                emb = pickle.load(f)
            self.weights['encoder_h'] = nn.Parameter(torch.FloatTensor(emb[0]))
            self.weights['decoder_h'] = nn.Parameter(torch.FloatTensor(emb[1]))
            self.biases['encoder_b'] = nn.Parameter(torch.FloatTensor(emb[2]))
            self.biases['decoder_b'] = nn.Parameter(torch.FloatTensor(emb[3]))

        self.d_params = [self.weights['encoder_h'], self.weights['decoder_h'],
                         self.biases['encoder_b'], self.biases['decoder_b']]

    def decoder(self, x):
        # Decoder Hidden layer with sigmoid activation #1
        layer = torch.sigmoid(torch.add(torch.matmul(x, self.weights['decoder_h'].t()), self.biases['decoder_b']))
        return layer

    def l2_loss(self):
        l2 = torch.nn.functional.l2_loss(self.weights['encoder_h']) + torch.nn.functional.l2_loss(
            self.biases['decoder_b']) + torch.nn.functional.l2_loss(self.biases['encoder_b']) + torch.nn.functional.l2_loss(
            self.weights['decoder_h'])
        return l2

In [15]:
conf = {
    'batch': 32,
    'n_input': 100,
    'hidden': 64,
    'lr': 0.001,
    'reg_lambda': 0.001,
    'initval': 'NULL',
    "keep_prob": 0.8,
    "input_keep_prob": (0.5,0.8),
    'save': './'
}
dae_model = DAE(conf)
dae_model.init_weight()

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(dae_model.parameters(), lr=conf['lr'])

In [17]:
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split


np.random.seed(42)  # Set the random seed for reproducibility

n = 500  # Number of unique songs
m = 50   # Number of playlists
k = 100  # Number of unique artists

# Generate random binary matrix P
P = np.random.randint(0, 2, size=(m, n))

# Generate random binary matrix A
A = np.random.randint(0, 2, size=(m, k))

A = torch.tensor(A)
P = torch.tensor(P)

class MyDataset(Dataset):
    def __init__(self, A):
        self.A = A

    def __len__(self):
        return len(self.A)

    def __getitem__(self, index):
        return self.A[index]

dataset = MyDataset(A)
train_ratio = 0.8
test_ratio = 0.2
batch_size = 32
train_size = int(train_ratio * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [18]:
num_epochs = 10
for epoch in range(num_epochs):
    for batch_data in train_loader:
        inputs, targets = batch_data
        optimizer.zero_grad()
        outputs = dae_model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

ValueError: ignored